In [37]:
import sys
from pathlib import Path

# Get the project root 
project_root = Path.cwd().parent

# Add project root to sys.path
sys.path.append(str(project_root))

In [38]:
import pandas as pd
import importlib

# Import and reload to get latest changes
import src.preprocessing as preprocessing
importlib.reload(preprocessing)

from src.preprocessing import (
    load_raw_2015,
    build_and_save_preprocessed,
    classify_variables,
)

In [39]:
# Step 1: Load raw data
df_raw = load_raw_2015()
print(f"Raw data shape: {df_raw.shape}")
print(f"Columns: {df_raw.shape[1]}")
df_raw.head()

Raw data shape: (441456, 171)
Columns: 171


,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,5.0,15.0,18,10.0,1,1,2.0,1.0,1.0,1.0,...,4,2,1.0,1.0,1.0,1,1,NaN,NaN,1.0
1,3.0,88.0,88,NaN,2,1,1.0,4.0,3.0,NaN,...,2,2,3.0,3.0,4.0,2,2,NaN,NaN,2.0
2,4.0,15.0,88,88.0,1,2,2.0,1.0,3.0,NaN,...,9,9,9.0,9.0,9.0,9,9,9.0,9.0,NaN
3,5.0,30.0,30,30.0,1,2,1.0,1.0,1.0,1.0,...,4,2,1.0,1.0,1.0,1,1,NaN,NaN,9.0
4,5.0,20.0,88,30.0,1,1,2.0,1.0,3.0,NaN,...,4,2,1.0,1.0,1.0,1,1,NaN,NaN,1.0


## Step 2: Apply Codebook Cleaning

BRFSS uses coded values for missing data (e.g., 7, 9, 77, 99, 777, 999). We need to convert these to proper NaN values so they're recognized as missing data in our analysis.

**Examples:**
- Yes/No questions: 1=Yes, 2=No, 7=Don't know, 9=Refused → Convert 7,9 to NaN
- Frequency questions: 555=Never, 777=Don't know, 999=Refused → Convert 777,999 to NaN, 555 to 0
- Days (0-30): 88=None, 77=Don't know, 99=Refused → Convert 88 to 0, 77,99 to NaN

In [40]:
from src.preprocessing import apply_codebook_cleaning

df_cleaned = apply_codebook_cleaning(df_raw)
print(f"After codebook cleaning: {df_cleaned.shape}")
print(f"\nMissing values before: {df_raw.isna().sum().sum():,}")
print(f"Missing values after: {df_cleaned.isna().sum().sum():,}")
print(f"Increase: {df_cleaned.isna().sum().sum() - df_raw.isna().sum().sum():,}")

After codebook cleaning: (441456, 171)

Missing values before: 9,167,996
Missing values after: 11,144,015
Increase: 1,976,019
Increase: 1,976,019


## Step 3: Drop Raw Columns When Calculated Versions Exist

BRFSS provides both raw and calculated versions of some variables. For example:
- `WEIGHT2` (raw weight in pounds) → `WTKG3` (calculated weight in kg)
- `HEIGHT3` (raw height) → `HTIN4`, `HTM4`, `_BMI5` (calculated height and BMI)
- `ALCDAY5` (raw alcohol frequency) → `_DRNKWEK` (calculated drinks per week)

**Why drop raw versions?**
- Avoid redundancy and multicollinearity
- Calculated versions are standardized and easier to use
- Reduces dimensionality

Let's check which columns will be dropped:

In [41]:
from src.preprocessing import RAW_TO_CALC, drop_raw_when_calc_exists

# Check which columns will be dropped
print("Raw columns that will be dropped if calculated versions exist:\n")
for raw_col, calc_cols in RAW_TO_CALC.items():
    if raw_col in df_cleaned.columns:
        existing_calc = [c for c in calc_cols if c in df_cleaned.columns]
        if existing_calc:
            print(f"✓ {raw_col} → {existing_calc}")

Raw columns that will be dropped if calculated versions exist:

✓ WEIGHT2 → ['WTKG3', '_BMI5']
✓ HEIGHT3 → ['HTIN4', 'HTM4', '_BMI5']
✓ ALCDAY5 → ['DROCDY3_', '_DRNKWEK', '_RFDRHV5', '_RFBING5']
✓ FRUITJU1 → ['FTJUDA1_']
✓ FRUIT1 → ['FRUTDA1_']
✓ FVBEANS → ['BEANDAY_']
✓ FVGREEN → ['GRENDAY_']
✓ FVORANG → ['ORNGDAY_']
✓ VEGETAB1 → ['VEGEDA1_']
✓ EXEROFT1 → ['PAFREQ1_']
✓ EXERHMM1 → ['PADUR1_']
✓ EXEROFT2 → ['PAFREQ2_']
✓ EXERHMM2 → ['PADUR2_']
✓ STRENGTH → ['STRFREQ_', '_PASTRNG']


In [42]:
# Apply the dropping
df_no_raw = drop_raw_when_calc_exists(df_cleaned)
print(f"\nBefore: {df_cleaned.shape[1]} columns")
print(f"After: {df_no_raw.shape[1]} columns")
print(f"Dropped: {df_cleaned.shape[1] - df_no_raw.shape[1]} columns")


Before: 171 columns
After: 157 columns
Dropped: 14 columns


## Step 4: Identify and Drop High-Missing Columns

Columns with >50% missing values provide little information and can harm model performance. Let's identify them first before deciding to drop.

In [43]:
from src.preprocessing import get_high_missing_columns, drop_columns

# Identify high-missing columns
high_missing_cols = get_high_missing_columns(df_no_raw, threshold=0.5)

print(f"Columns with >50% missing values ({len(high_missing_cols)}):\n")
missing_ratio = df_no_raw[high_missing_cols].isna().mean().sort_values(ascending=False)
for col, ratio in missing_ratio.items():
    print(f"  {col}: {ratio*100:.1f}% missing")

Columns with >50% missing values (17):

  _CHISPNC: 86.0% missing
  JOINPAIN: 70.3% missing
  ARTHDIS2: 70.0% missing
  _PNEUMO2: 69.5% missing
  ARTHSOCL: 69.4% missing
  LMTJOIN3: 69.4% missing
  _FLSHOT6: 68.5% missing
  TRNSGNDR: 62.8% missing
  BPMEDS: 59.7% missing
  FLSHTMY2: 59.5% missing
  SMOKDAY2: 58.4% missing
  IMFVPLAC: 56.8% missing
  PADUR2_: 56.4% missing
  PAFREQ2_: 55.9% missing
  MAXDRNKS: 53.9% missing
  AVEDRNK2: 53.1% missing
  DRNK3GE5: 53.0% missing


In [44]:
# Drop high-missing columns
df_no_high_missing = drop_columns(df_no_raw, high_missing_cols, verbose=True)
print(f"\n✓ Kept {df_no_high_missing.shape[1]} columns")

Dropping 17 columns:
  - _CHISPNC
  - JOINPAIN
  - ARTHDIS2
  - _PNEUMO2
  - ARTHSOCL
  - LMTJOIN3
  - _FLSHOT6
  - TRNSGNDR
  - BPMEDS
  - FLSHTMY2
  - SMOKDAY2
  - IMFVPLAC
  - PADUR2_
  - PAFREQ2_
  - MAXDRNKS
  - AVEDRNK2
  - DRNK3GE5

✓ Kept 140 columns

✓ Kept 140 columns


## Step 5: Drop Questionnaire Metadata Columns

Columns like questionnaire version (`QSTVER`) and interview language (`QSTLANG`) are administrative metadata, not health data. They don't contribute to predicting health outcomes.

In [45]:
from src.preprocessing import drop_questionnaire_metadata_columns

df_no_metadata = drop_questionnaire_metadata_columns(df_no_high_missing)
print(f"Columns remaining: {df_no_metadata.shape[1]}")


Dropping 2 questionnaire metadata columns: ['QSTVER', 'QSTLANG']
Columns remaining: 138


## Step 6: Drop High-Cardinality Text Columns

Free-text columns with many unique values (like "other activity type") are difficult to use in models and typically provide little predictive value.

In [46]:
from src.preprocessing import drop_high_cardinality_text_columns

df_no_text = drop_high_cardinality_text_columns(df_no_metadata, threshold=50)
print(f"Columns remaining: {df_no_text.shape[1]}")


Dropping 2 high-cardinality text columns (>50 unique values):
  - EXACTOT1 (5642 unique values)
  - EXACTOT2 (7623 unique values)
Columns remaining: 136
Columns remaining: 136


## Step 7: Fix Boolean Encoding

BRFSS encodes Yes/No questions as 1=Yes, 2=No. For modeling, we need 0=No, 1=Yes (standard binary encoding).

Also, some columns have scientific notation issues (5.4e-79 instead of 0).

In [47]:
from src.preprocessing import fix_boolean_encoding

# Check before
print("Before encoding fix (sample boolean columns):")
bool_sample = [col for col in df_no_text.columns if df_no_text[col].dropna().nunique() == 2][:3]
for col in bool_sample:
    print(f"  {col}: {sorted(df_no_text[col].dropna().unique())}")

df_fixed = fix_boolean_encoding(df_no_text)

# Check after
print("\nAfter encoding fix:")
for col in bool_sample:
    print(f"  {col}: {sorted(df_fixed[col].dropna().unique())}")

Before encoding fix (sample boolean columns):
  HLTHPLN1: [np.float64(1.0), np.float64(2.0)]
  MEDCOST: [np.float64(1.0), np.float64(2.0)]
  BLOODCHO: [np.float64(1.0), np.float64(2.0)]
  HLTHPLN1: [np.float64(1.0), np.float64(2.0)]
  MEDCOST: [np.float64(1.0), np.float64(2.0)]
  BLOODCHO: [np.float64(1.0), np.float64(2.0)]

Mapping 1->0, 2->1 for 55 boolean columns

Mapping 1->0, 2->1 for 55 boolean columns
✓ Fixed encoding for 60 boolean columns

After encoding fix:
  HLTHPLN1: [np.float64(0.0), np.float64(1.0)]
  MEDCOST: [np.float64(0.0), np.float64(1.0)]
  BLOODCHO: [np.float64(0.0), np.float64(1.0)]
✓ Fixed encoding for 60 boolean columns

After encoding fix:
  HLTHPLN1: [np.float64(0.0), np.float64(1.0)]
  MEDCOST: [np.float64(0.0), np.float64(1.0)]
  BLOODCHO: [np.float64(0.0), np.float64(1.0)]


## Step 8: Drop Rows with Missing Target Variable

For supervised learning, we need the target variable (`_MICHD` - cardiovascular disease indicator). Rows without this value cannot be used for training or evaluation.

In [48]:
# Define target variable
TARGET_COL = '_MICHD'

# Check missing target values
print(f"Before dropping missing target:")
print(f"  Total rows: {df_fixed.shape[0]:,}")
print(f"  Rows with missing target: {df_fixed[TARGET_COL].isna().sum():,}")
print(f"  Percentage missing: {df_fixed[TARGET_COL].isna().mean()*100:.2f}%")

# Drop rows with missing target
df_final = df_fixed.dropna(subset=[TARGET_COL])

print(f"\nAfter dropping missing target:")
print(f"  Total rows: {df_final.shape[0]:,}")
print(f"  Rows dropped: {df_fixed.shape[0] - df_final.shape[0]:,}")

# Update df_pre
df_pre = df_final

Before dropping missing target:
  Total rows: 441,456
  Rows with missing target: 3,942
  Percentage missing: 0.89%

After dropping missing target:
  Total rows: 437,514
  Rows dropped: 3,942


## Step 9: Classify Variables and Save Final Dataset

Now classify columns into numeric, categorical, and boolean types, then save the final preprocessed dataset.

In [49]:
from src.preprocessing import save_preprocessed_csv

# Classify variables
numeric_cols, categorical_cols, boolean_cols = classify_variables(df_pre)

# Save
save_path = save_preprocessed_csv(df_pre, "clean_2015.csv")

print(f"\n✓ Final preprocessed dataset: {df_pre.shape}")
print(f"✓ Saved to: {save_path}")


=== Variable Classification ===
Numeric: 42
Categorical: 34
Boolean: 60
Saved cleaned dataset to: D:\dataprep_final\DataPrepPrj\data\processed\clean_2015.csv

✓ Final preprocessed dataset: (437514, 136)
✓ Saved to: D:\dataprep_final\DataPrepPrj\data\processed\clean_2015.csv
Saved cleaned dataset to: D:\dataprep_final\DataPrepPrj\data\processed\clean_2015.csv

✓ Final preprocessed dataset: (437514, 136)
✓ Saved to: D:\dataprep_final\DataPrepPrj\data\processed\clean_2015.csv


In [53]:
# Summary of final preprocessed data
print(f"\n{'='*50}")
print(f"FINAL PREPROCESSED DATASET SUMMARY")
print(f"{'='*50}")
print(f"Total columns: {df_pre.shape[1]}")
print(f"Total rows: {df_pre.shape[0]}")
print(f"\nVariable breakdown:")
print(f"  - Numeric: {len(numeric_cols)}")
print(f"  - Categorical: {len(categorical_cols)}")
print(f"  - Boolean: {len(boolean_cols)}")
print(f"\nMissing values: {df_pre.isna().sum().sum():,} ({df_pre.isna().sum().sum() / df_pre.size * 100:.2f}%)")


FINAL PREPROCESSED DATASET SUMMARY
Total columns: 136
Total rows: 437514

Variable breakdown:
  - Numeric: 42
  - Categorical: 34
  - Boolean: 60

Missing values: 5,177,882 (8.70%)
